In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145674 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.17-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.17-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.17-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image as img
from keras.models import Model
import numpy as np
import os
import pickle
import time

In [0]:
captions = None
with open('/content/drive/Image Captioning/Utility Data/captions.txt') as f:
  captions = f.read()
captions = eval(captions)

In [5]:
split_factor = 0.90
split = int(split_factor*len(captions))
train_captions = dict(list(captions.items())[:split])
test_captions = dict(list(captions.items())[split:])
print('Number of training examples: ', len(train_captions))
print('Number of testing examples: ', len(test_captions))

Number of training examples:  7282
Number of testing examples:  810


In [0]:
for k in train_captions.keys():
  for i in range(len(train_captions[k])):
    train_captions[k][i] = '<start> ' + train_captions[k][i] + ' <end>'

In [0]:
with open('/content/drive/Image Captioning/Utility Data/train_captions.txt', 'w') as f:
  f.write(str(train_captions))
with open('/content/drive/Image Captioning/Utility Data/test_captions.txt', 'w') as f:
  f.write(str(test_captions))

In [8]:
resnet = ResNet50(weights = 'imagenet', input_shape = (224, 224, 3))
resnet.summary()














102858752/102853048 [==============================] - 4s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________

In [9]:
model = Model(inputs = resnet.input, outputs = resnet.layers[-2].output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
def preprocess(path, captions):
  preprocessed_images = {}
  cnt = 0
  for image_id in captions.keys():
    image_path = os.path.join(path, image_id + '.jpg')
    try:
      image = img.load_img(image_path, target_size = (224, 224))
    except:
      continue
    image = img.img_to_array(image)
    image = preprocess_input(image)
    image = np.expand_dims(image, axis = 0)
    image = model.predict(image)
    image = image.reshape((-1, ))
    preprocessed_images[image_id] = image
    if cnt % 100 == 0:
      print('|', end = '')
    cnt += 1
  print(' Done.')
  return preprocessed_images

In [25]:
path = '/content/drive/Image Captioning/Dataset/Flicker8k_Dataset'
start_t = time.time()
train_images = preprocess(path, train_captions)
end_t = time.time()
print('Time taken: ', end_t - start_t)

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| Done.
Time taken:  4167.189735651016


In [0]:
with open('/content/drive/Image Captioning/Utility Data/train_images.pkl', 'wb') as f:
  pickle.dump(train_images, f)

In [27]:
path = '/content/drive/Image Captioning/Dataset/Flicker8k_Dataset'
test_images = preprocess(path, test_captions)

||||||||| Done.


In [0]:
with open('/content/drive/Image Captioning/Utility Data/test_images.pkl', 'wb') as f:
  pickle.dump(test_images, f)